# Expression Recognition

# 1. Importing, exploring and cleaning ISEAR dataset

In [1]:
!git clone https://github.com/Bhattars1/MSc-Project-Expression-Recognition-

Cloning into 'MSc-Project-Expression-Recognition-'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 33 (delta 9), reused 16 (delta 2), pack-reused 0
Receiving objects: 100% (33/33), 4.72 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [24]:
import pandas as pd
def upload_file(path):
  df = pd.read_excel(path, engine='openpyxl')
  print(df.head())
  print(df.info())
  return df

In [25]:
file_path = '/content/MSc-Project-Expression-Recognition-/DATA.xlsx'
df = upload_file(file_path)

       ID  CITY  COUN  SUBJ  SEX  AGE  RELI  PRAC  FOCC  MOCC  ...  SELF  \
0  101032     1    10    32    2   21     2     2     7     1  ...     3   
1  101040     1    10    40    2   18     2     2     7     7  ...     1   
2  261050     1    26    50    1   23     4     1     9     1  ...     0   
3  101039     1    10    39    1   19     2     2     8     1  ...     0   
4  101038     1    10    38    2   18     2     2     7     6  ...     0   

   RELA  VERBAL  NEUTRO    MYKEY  Field3  Field2  Field1  \
0     0       3       0  1010323       2       2   anger   
1     0       0       1  1010403       4       3   anger   
2     0       0       1  2610503       4       3   anger   
3     1       3       1  1010393       4       4   anger   
4     1       3       2  1010383       3       4   anger   

                                                 SIT  STATE  
0  When a boy tried to fool me so he would be OK ...      1  
1  I felt anger when I saw that I was being misle...      

In [29]:
def filter_data(df, selected_columns):
  df_selected = df[selected_columns]
  print(df_selected.head(3))
  return df_selected

In [30]:
selected_columns = ["COUN", "SEX", "RELI", "PRAC", "FIEL", "Field1", "SIT"]
df_selected = filter_data(df, selected_columns)

   COUN  SEX  RELI  PRAC  FIEL Field1  \
0    10    2     2     2     2  anger   
1    10    2     2     2     9  anger   
2    26    1     4     1     1  anger   

                                                 SIT  
0  When a boy tried to fool me so he would be OK ...  
1  I felt anger when I saw that I was being misle...  
2  Once a friend had pushed me and I had fallen o...  


In [33]:
def check_unique_values(df, selected_columns):
  for i in selected_columns[:-1]:
    unique = set(df[i])
    print(f"There are {len(unique)} unique values in {i}")
    print(f"They are listed below:\n{list(unique)}\n")
check_unique_values(df_selected, selected_columns)

There are 16 unique values in COUN
They are listed below:
[33, 1, 2, 4, 8, 9, 10, 12, 16, 17, 19, 20, 22, 26, 27, 30]

There are 3 unique values in SEX
They are listed below:
[0, 1, 2]

There are 9 unique values in RELI
They are listed below:
[0, 1, 2, 3, 4, 5, 6, 7, 8]

There are 3 unique values in PRAC
They are listed below:
[0, 1, 2]

There are 10 unique values in FIEL
They are listed below:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

There are 7 unique values in Field1
They are listed below:
['guilt', 'sadness', 'joy', 'disgust', 'fear', 'anger', 'shame']



In [34]:
def data_and_labels(df, data, labels):
  data = df[data]
  labels = df[labels]
  return data, labels
data_column = "SIT"
label_column = "Field1"
data, labels = data_and_labels(df_selected, data_column, label_column)

In [42]:
len(labels), len(data)

(7666, 7666)

In [49]:
def check_unique_values(data):
  unique_expressions = list(set(labels))
  for i in unique_expressions:
    print(f"{i} : {list(labels).count(i)}")
check_unique_values(labels)

guilt : 1093
sadness : 1096
joy : 1094
disgust : 1096
fear : 1095
anger : 1096
shame : 1096
